# Week 2, Lab 4: Knowledge-Based Systems and Applications

## Welcome to the Final Lab!

You've learned the theory - now let's build real knowledge-based systems! We'll use professional libraries and create practical applications.

### What You'll Learn

- Expert system architecture
- Using SymPy for logic
- Knowledge graphs and semantic networks
- Building a medical diagnosis system
- Real-world applications

### Real-World Applications

- MYCIN (medical diagnosis)
- DENDRAL (chemical analysis)
- Cyc (commonsense knowledge)
- Google Knowledge Graph
- Watson (IBM)

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sympy import symbols, And, Or, Not, Implies, satisfiable
from sympy.logic.boolalg import to_cnf
from typing import List, Dict, Set, Tuple, Optional
import pandas as pd

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print(f"SymPy is ready for symbolic reasoning!")

## 1. Using SymPy for Logic

**SymPy** is a Python library for symbolic mathematics, including logic!

### Features:
- Symbolic logical expressions
- Satisfiability (SAT) solving
- CNF conversion
- Simplification
- Truth table generation

In [ ]:
# Create symbolic variables
P, Q, R = symbols('P Q R')

# Create logical expressions
expr1 = Implies(P, Q)  # P → Q
expr2 = And(P, Q)      # P ∧ Q
expr3 = Or(Not(P), R)  # ¬P ∨ R

print("SymPy Logical Expressions:")
print("=" * 50)
print(f"1. P → Q: {expr1}")
print(f"2. P ∧ Q: {expr2}")
print(f"3. ¬P ∨ R: {expr3}")

# Combine expressions
complex_expr = And(Implies(P, Q), P, Not(Q))
print(f"\n4. (P → Q) ∧ P ∧ ¬Q: {complex_expr}")

# Check satisfiability
result = satisfiable(complex_expr)
print(f"\nIs it satisfiable? {result}")
print("→ This is a contradiction! (Modus Ponens violation)")

In [ ]:
# Convert to CNF (Conjunctive Normal Form)
print("CNF Conversion:")
print("=" * 50)

# Example: (P → Q) ∧ (Q → R)
formula = And(Implies(P, Q), Implies(Q, R))
print(f"\nOriginal: {formula}")

cnf_formula = to_cnf(formula)
print(f"CNF: {cnf_formula}")

# Check if (P → Q) ∧ (Q → R) entails (P → R)
kb = And(Implies(P, Q), Implies(Q, R))
query = Implies(P, R)

# KB entails query if KB ∧ ¬query is unsatisfiable
test = And(kb, Not(query))
result = satisfiable(test)

print(f"\nDoes KB entail query?")
print(f"  KB: {kb}")
print(f"  Query: {query}")
print(f"  KB ∧ ¬Query is satisfiable: {result}")
if result == False:
    print("  ✓ YES! KB entails query (transitivity of implication)")
else:
    print(f"  ✗ NO. Counterexample: {result}")

## 2. Expert System Architecture

An **expert system** has three main components:

1. **Knowledge Base**: Facts and rules (what the expert knows)
2. **Inference Engine**: Reasoning mechanism (how to apply knowledge)
3. **User Interface**: How users interact with the system

### Classic Expert Systems:
- **MYCIN**: Medical diagnosis (1970s)
- **DENDRAL**: Chemical structure analysis
- **XCON**: Computer configuration

Let's build a simple medical diagnosis system!

In [ ]:
class ExpertSystem:
    """Simple rule-based expert system."""
    
    def __init__(self, name: str):
        self.name = name
        self.facts = set()
        self.rules = []
        self.conclusions = set()
    
    def add_fact(self, fact: str):
        """Add a known fact."""
        self.facts.add(fact)
    
    def add_rule(self, conditions: List[str], conclusion: str, confidence: float = 1.0):
        """
        Add a rule with conditions and conclusion.
        
        Args:
            conditions: List of required facts
            conclusion: What to conclude if conditions met
            confidence: Confidence level (0-1)
        """
        self.rules.append({
            'conditions': conditions,
            'conclusion': conclusion,
            'confidence': confidence
        })
    
    def infer(self, verbose: bool = True) -> Set[Tuple[str, float]]:
        """
        Run inference to derive conclusions.
        
        Returns:
            Set of (conclusion, confidence) tuples
        """
        if verbose:
            print(f"\n{self.name} - Running Inference")
            print("=" * 60)
            print(f"\nKnown facts: {len(self.facts)}")
            for fact in sorted(self.facts):
                print(f"  ✓ {fact}")
        
        conclusions = set()
        
        # Check each rule
        for rule in self.rules:
            # Check if all conditions are satisfied
            if all(cond in self.facts for cond in rule['conditions']):
                conclusion = (rule['conclusion'], rule['confidence'])
                conclusions.add(conclusion)
                
                if verbose:
                    print(f"\n  → Conclusion: {rule['conclusion']}")
                    print(f"    Confidence: {rule['confidence']*100:.0f}%")
                    print(f"    Based on: {', '.join(rule['conditions'])}")
        
        self.conclusions = conclusions
        return conclusions
    
    def get_diagnosis(self) -> List[Tuple[str, float]]:
        """Get diagnoses sorted by confidence."""
        return sorted(self.conclusions, key=lambda x: x[1], reverse=True)

# Build a simple medical diagnosis system
medical_expert = ExpertSystem("Medical Diagnosis System")

# Add rules for flu
medical_expert.add_rule(
    ['fever', 'cough', 'body_aches'],
    'Influenza (Flu)',
    confidence=0.85
)

medical_expert.add_rule(
    ['fever', 'sore_throat', 'cough'],
    'Common Cold',
    confidence=0.75
)

medical_expert.add_rule(
    ['fever', 'headache', 'nausea'],
    'Viral Infection',
    confidence=0.70
)

medical_expert.add_rule(
    ['fever', 'cough', 'shortness_of_breath'],
    'Pneumonia',
    confidence=0.80
)

medical_expert.add_rule(
    ['rash', 'fever', 'joint_pain'],
    'Viral Rash Illness',
    confidence=0.65
)

print("Medical Expert System Created!")
print(f"Loaded {len(medical_expert.rules)} diagnostic rules")

In [ ]:
# Example 1: Patient with flu symptoms
print("\nPatient 1: Flu-like Symptoms")
print("=" * 60)

patient1 = ExpertSystem("Patient 1 Diagnosis")
patient1.rules = medical_expert.rules

# Patient symptoms
patient1.add_fact('fever')
patient1.add_fact('cough')
patient1.add_fact('body_aches')

# Run diagnosis
patient1.infer()

print("\nDiagnoses (sorted by confidence):")
for diagnosis, confidence in patient1.get_diagnosis():
    print(f"  {confidence*100:.0f}% - {diagnosis}")

In [ ]:
# Example 2: Patient with respiratory symptoms
print("\nPatient 2: Respiratory Symptoms")
print("=" * 60)

patient2 = ExpertSystem("Patient 2 Diagnosis")
patient2.rules = medical_expert.rules

# Different symptoms
patient2.add_fact('fever')
patient2.add_fact('cough')
patient2.add_fact('shortness_of_breath')

# Run diagnosis
patient2.infer()

print("\nDiagnoses (sorted by confidence):")
for diagnosis, confidence in patient2.get_diagnosis():
    print(f"  {confidence*100:.0f}% - {diagnosis}")
    
print("\n⚠ Note: This is a simplified demonstration.")
print("   Real medical systems use much more complex reasoning!")

## 3. Knowledge Graphs and Semantic Networks

**Knowledge graphs** represent knowledge as a network of entities and relationships.

### Structure:
- **Nodes**: Entities (people, places, things)
- **Edges**: Relationships (is-a, part-of, related-to)
- **Properties**: Attributes of entities

### Examples:
- Google Knowledge Graph
- Facebook Social Graph
- DBpedia / Wikipedia
- Wikidata

In [ ]:
# Build a simple knowledge graph
class KnowledgeGraph:
    """Simple knowledge graph using NetworkX."""
    
    def __init__(self):
        self.graph = nx.MultiDiGraph()  # Allows multiple edges
    
    def add_entity(self, entity: str, entity_type: str = None, **properties):
        """Add an entity to the knowledge graph."""
        self.graph.add_node(entity, type=entity_type, **properties)
    
    def add_relation(self, subject: str, relation: str, object: str, **properties):
        """Add a relationship between entities."""
        self.graph.add_edge(subject, object, relation=relation, **properties)
    
    def query(self, subject: str = None, relation: str = None, object: str = None):
        """Query the knowledge graph."""
        results = []
        
        for s, o, data in self.graph.edges(data=True):
            # Check if matches query
            if ((subject is None or s == subject) and
                (relation is None or data.get('relation') == relation) and
                (object is None or o == object)):
                results.append((s, data.get('relation'), o))
        
        return results
    
    def visualize(self, title: str = "Knowledge Graph"):
        """Visualize the knowledge graph."""
        plt.figure(figsize=(14, 10))
        
        pos = nx.spring_layout(self.graph, k=2, iterations=50, seed=42)
        
        # Draw nodes
        nx.draw_networkx_nodes(self.graph, pos, node_color='lightblue',
                              node_size=3000, alpha=0.9)
        nx.draw_networkx_labels(self.graph, pos, font_size=10, font_weight='bold')
        
        # Draw edges with labels
        for (u, v, key, data) in self.graph.edges(keys=True, data=True):
            nx.draw_networkx_edges(self.graph, pos, [(u, v)],
                                  width=2, alpha=0.6, arrows=True,
                                  arrowsize=20, arrowstyle='->')
        
        # Draw edge labels
        edge_labels = {(u, v): data['relation'] 
                      for u, v, data in self.graph.edges(data=True)}
        nx.draw_networkx_edge_labels(self.graph, pos, edge_labels, 
                                    font_size=9, font_color='red')
        
        plt.title(title, fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Build a knowledge graph about AI
kg = KnowledgeGraph()

# Add entities
kg.add_entity("Artificial Intelligence", entity_type="Field")
kg.add_entity("Machine Learning", entity_type="Field")
kg.add_entity("Deep Learning", entity_type="Field")
kg.add_entity("Neural Networks", entity_type="Technique")
kg.add_entity("Search Algorithms", entity_type="Technique")
kg.add_entity("Logic", entity_type="Technique")
kg.add_entity("Python", entity_type="Language")
kg.add_entity("TensorFlow", entity_type="Library")

# Add relationships
kg.add_relation("Machine Learning", "is_subfield_of", "Artificial Intelligence")
kg.add_relation("Deep Learning", "is_subfield_of", "Machine Learning")
kg.add_relation("Neural Networks", "used_in", "Deep Learning")
kg.add_relation("Search Algorithms", "part_of", "Artificial Intelligence")
kg.add_relation("Logic", "part_of", "Artificial Intelligence")
kg.add_relation("Python", "used_for", "Machine Learning")
kg.add_relation("TensorFlow", "written_in", "Python")
kg.add_relation("TensorFlow", "used_for", "Deep Learning")

print("Knowledge Graph Created!")
print(f"  Entities: {kg.graph.number_of_nodes()}")
print(f"  Relations: {kg.graph.number_of_edges()}")

kg.visualize("AI Knowledge Graph")

In [ ]:
# Query the knowledge graph
print("Knowledge Graph Queries:")
print("=" * 60)

# Query 1: What are subfields of AI?
print("\n1. What is a subfield of Artificial Intelligence?")
results = kg.query(relation="is_subfield_of", object="Artificial Intelligence")
for s, r, o in results:
    print(f"   {s}")

# Query 2: What is Machine Learning related to?
print("\n2. What is Machine Learning related to?")
results = kg.query(subject="Machine Learning")
for s, r, o in results:
    print(f"   {r} → {o}")

# Query 3: What is Python used for?
print("\n3. What is Python used for?")
results = kg.query(subject="Python", relation="used_for")
for s, r, o in results:
    print(f"   {o}")

# Query 4: What uses Neural Networks?
print("\n4. What uses Neural Networks?")
results = kg.query(object="Neural Networks")
for s, r, o in results:
    print(f"   {s} ({r})")

## 4. Putting It All Together: Enhanced Expert System

Let's combine everything into a more sophisticated system!

In [ ]:
class AdvancedExpertSystem:
    """Advanced expert system with explanation capability."""
    
    def __init__(self, name: str):
        self.name = name
        self.facts = {}
        self.rules = []
        self.inference_chain = []
    
    def add_fact(self, fact: str, certainty: float = 1.0, source: str = "user"):
        """Add a fact with certainty factor."""
        self.facts[fact] = {'certainty': certainty, 'source': source}
    
    def add_rule(self, rule_id: str, conditions: List[str], 
                conclusion: str, certainty: float = 1.0, explanation: str = ""):
        """Add a rule with explanation."""
        self.rules.append({
            'id': rule_id,
            'conditions': conditions,
            'conclusion': conclusion,
            'certainty': certainty,
            'explanation': explanation
        })
    
    def infer(self, max_iterations: int = 10) -> Dict[str, Dict]:
        """Run inference with explanation tracking."""
        print(f"\n{self.name}")
        print("=" * 60)
        
        for iteration in range(max_iterations):
            new_facts = {}
            
            for rule in self.rules:
                # Check if all conditions are satisfied
                if all(cond in self.facts for cond in rule['conditions']):
                    conclusion = rule['conclusion']
                    
                    # Skip if already known
                    if conclusion in self.facts:
                        continue
                    
                    # Calculate combined certainty
                    condition_certainties = [self.facts[c]['certainty'] 
                                           for c in rule['conditions']]
                    combined_certainty = min(condition_certainties) * rule['certainty']
                    
                    new_facts[conclusion] = {
                        'certainty': combined_certainty,
                        'source': f"Rule {rule['id']}"
                    }
                    
                    # Track inference
                    self.inference_chain.append({
                        'iteration': iteration + 1,
                        'rule': rule['id'],
                        'conditions': rule['conditions'],
                        'conclusion': conclusion,
                        'certainty': combined_certainty,
                        'explanation': rule['explanation']
                    })
                    
                    print(f"\nIteration {iteration + 1}:")
                    print(f"  Rule {rule['id']}: {conclusion}")
                    print(f"  Certainty: {combined_certainty*100:.0f}%")
                    print(f"  Because: {', '.join(rule['conditions'])}")
                    if rule['explanation']:
                        print(f"  Explanation: {rule['explanation']}")
            
            if not new_facts:
                print(f"\n✓ Inference complete after {iteration + 1} iterations")
                break
            
            self.facts.update(new_facts)
        
        return self.facts
    
    def explain(self, fact: str):
        """Explain how a fact was derived."""
        print(f"\nExplanation for: {fact}")
        print("=" * 60)
        
        if fact not in self.facts:
            print("  This fact is not known.")
            return
        
        fact_info = self.facts[fact]
        print(f"  Certainty: {fact_info['certainty']*100:.0f}%")
        print(f"  Source: {fact_info['source']}")
        
        # Find in inference chain
        for step in self.inference_chain:
            if step['conclusion'] == fact:
                print(f"\n  Derived by Rule {step['rule']}:")
                print(f"    Conditions: {', '.join(step['conditions'])}")
                if step['explanation']:
                    print(f"    Reasoning: {step['explanation']}")

# Build an advanced diagnostic system
advanced_system = AdvancedExpertSystem("Advanced Medical Diagnosis")

# Add symptoms (facts)
advanced_system.add_fact('high_fever', certainty=0.95, source='measurement')
advanced_system.add_fact('persistent_cough', certainty=0.90, source='observation')
advanced_system.add_fact('fatigue', certainty=0.85, source='patient_report')

# Add diagnostic rules
advanced_system.add_rule(
    rule_id="R1",
    conditions=['high_fever', 'persistent_cough'],
    conclusion='respiratory_infection',
    certainty=0.80,
    explanation="High fever combined with persistent cough indicates respiratory tract infection"
)

advanced_system.add_rule(
    rule_id="R2",
    conditions=['respiratory_infection', 'fatigue'],
    conclusion='possible_pneumonia',
    certainty=0.70,
    explanation="Respiratory infection with significant fatigue suggests possible pneumonia"
)

advanced_system.add_rule(
    rule_id="R3",
    conditions=['possible_pneumonia'],
    conclusion='recommend_chest_xray',
    certainty=1.0,
    explanation="Suspected pneumonia requires chest X-ray for confirmation"
)

# Run inference
results = advanced_system.infer()

# Show all conclusions
print("\n" + "=" * 60)
print("All Derived Facts:")
print("=" * 60)
for fact, info in sorted(results.items(), key=lambda x: x[1]['certainty'], reverse=True):
    print(f"  {fact}")
    print(f"    Certainty: {info['certainty']*100:.0f}%")
    print(f"    Source: {info['source']}")

# Explain a specific conclusion
advanced_system.explain('recommend_chest_xray')

## 5. Key Takeaways

### Week 2 Complete Summary:

#### Lab 1: Propositional Logic
- Logical connectives (AND, OR, NOT, IMPLIES, IFF)
- Truth tables
- Knowledge bases
- Logic puzzles

#### Lab 2: Inference
- Inference rules (Modus Ponens, Resolution)
- Model checking
- CNF and Resolution algorithm
- Forward chaining

#### Lab 3: First-Order Logic
- Predicates and quantifiers
- Unification algorithm
- Reasoning with variables
- Family tree relationships

#### Lab 4: Applications
- Expert systems
- SymPy for logic
- Knowledge graphs
- Real-world systems

### Tools We Learned:

1. **SymPy**: Professional logic library
2. **NetworkX**: Knowledge graphs
3. **Custom engines**: Inference, unification

### Real-World Impact:

- **MYCIN**: 69% accuracy vs 65% for doctors
- **DENDRAL**: First expert system (1965)
- **Google Knowledge Graph**: 500+ billion facts
- **Watson**: Won Jeopardy! (2011)

### When to Use Knowledge-Based Systems:

✓ **Good for:**
- Rule-based domains
- Explainable AI needed
- Expert knowledge available
- Symbolic reasoning

✗ **Not good for:**
- Pattern recognition
- Learning from data
- Fuzzy/uncertain rules
- High-dimensional data

## Next Week Preview

**Week 3: Uncertainty** - Learn how to reason with probabilities:
- Bayes' theorem
- Bayesian networks
- Probabilistic reasoning
- Markov models

## Practice Exercises

1. Build an expert system for a domain you know (cooking, cars, sports)
2. Create a knowledge graph for your family or organization
3. Implement backward chaining for the expert system
4. Add certainty factors and combine evidence
5. Build a natural language interface for queries

## Final Project Ideas

1. **Troubleshooting System**: Computer/car problem diagnosis
2. **Recipe Recommender**: Based on ingredients and preferences
3. **Career Advisor**: Match skills to job recommendations
4. **Legal Reasoner**: Analyze simple legal scenarios
5. **Game Strategist**: Chess/game strategy advisor

---

**Congratulations on completing Week 2! 🎉**

You can now:
- Represent knowledge formally
- Build reasoning systems
- Create expert systems
- Work with knowledge graphs

This is the foundation of symbolic AI - the original vision of artificial intelligence! 🧠✨